This script simulates the U of A pattern

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import time
from IPython import display
import pandas as pd

from dedalus import public as de
from dedalus.extras import flow_tools

import logging
logger = logging.getLogger(__name__)

In [ ]:
#Aspect ratio 2
Lx, Ly = (112, 38)
nx, ny = (900, 350)

# Create bases and domain
x_basis = de.Fourier('x', nx, interval=(0, Lx), dealias=3/2)
y_basis = de.Fourier('y', ny, interval=(0, Ly), dealias=3/2)
domain = de.Domain([x_basis, y_basis], grid_dtype=np.float64)

In [ ]:
def grow_fun(A,B):
    F = (A.data)*(1-B.data)
    F[A.data < -1e-3] = 0
    return F


def grow_operator(field1,field2):
    return de.operators.GeneralFunction(
        field1.domain,
        layout = 'g',
        func = grow_fun,
        args = (field1,field2,)
    )

de.operators.parseables['grow'] = grow_operator

In [ ]:
params = np.load('Parameters.npz')
Diff = params['Diff'].flat[0]
K =  params['K'].flat[0]
g = params['g'].flat[0]

problem = de.IVP(domain, variables=['rho1','rho2','rho3','rho4','rhon','rhom'])
problem.parameters['D0'] = Diff
problem.parameters['K'] = K
problem.parameters['g0'] = g


problem.substitutions['rho_b'] = "rho1+rho2+rho3+rho4+rhom + rhon" 
problem.substitutions["Lap(A)"] = "dx(dx(A)) + dy(dy(A))"

problem.add_equation("dt(rho1)  - D0*Lap(rho1)  = g0*grow(rho1,rho_b) - K*rho1*rho2")
problem.add_equation("dt(rho2)  - D0*Lap(rho2)  = g0*grow(rho2,rho_b) - K*rho2*rho1")
problem.add_equation("dt(rho3)  - D0*Lap(rho3)  = g0*grow(rho3,rho_b) - K*rho3*rho4")
problem.add_equation("dt(rho4)  - D0*Lap(rho4)  = g0*grow(rho4,rho_b) - K*rho4*rho3")
problem.add_equation("dt(rhon)  - D0*Lap(rhon)  = g0*grow(rhon,rho_b)")

problem.add_equation("dt(rhom)  = 2*K*(rho1*rho2 + rho3*rho4 )")

In [ ]:
ts = de.timesteppers.RK443
solver =  problem.build_solver(ts)

In [ ]:
seed_pts = pd.read_csv('uofa_positions.csv',header=None)
seed_col = pd.read_csv('uofa_coloring.csv',header=None)

In [ ]:
x = domain.grid(0)
y = domain.grid(1)
rho1 = solver.state['rho1']
rho2 = solver.state['rho2']
rho3 = solver.state['rho3']
rho4 = solver.state['rho4']
rhon = solver.state['rhon']

def r(x,y,x0,y0):
    return np.sqrt((x-x0)**2+(y-y0)**2)

def blob_np(x,y,x0,y0,r0,w):
    return 0.5*(1 - np.tanh((r(x,y,x0,y0)-r0)/w))

def blob(x0,y0):
    r0 = 0.75
    w = 0.22
    return blob_np(x,y,x0 - np.min(seed_pts)[0] + 2,Ly - (y0- np.min(seed_pts)[1] + 2),r0,w)

for k in range(len(seed_pts)):
    for c in list(seed_col.loc[k,:]):
        if c == 'r':
            rho1['g'] = rho1['g'] + blob(seed_pts.loc[k,0],seed_pts.loc[k,1])

        if c == 'g':
            rho2['g'] = rho2['g'] + blob(seed_pts.loc[k,0],seed_pts.loc[k,1])

        if c == 'y':
            rho3['g'] = rho3['g'] + blob(seed_pts.loc[k,0],seed_pts.loc[k,1]) 

        if c == 'b':
            rho4['g'] = rho4['g'] + blob(seed_pts.loc[k,0],seed_pts.loc[k,1])
            
        if c == 'n':
            rhon['g'] = rhon['g'] + blob(seed_pts.loc[k,0],seed_pts.loc[k,1])         
            

In [ ]:
solver.stop_sim_time = 200
dt = 0.4*Lx/nx

In [ ]:
analysis = solver.evaluator.add_file_handler('digital', sim_dt=1, max_writes=900)
analysis.add_task('rho1')
analysis.add_task('rho2')
analysis.add_task('rho3')
analysis.add_task('rho4')
analysis.add_task('rhon')
analysis.add_task('rhom')

In [ ]:
logger.info('Starting loop')
start_time = time.time()
while solver.ok:
    solver.step(dt)
    if solver.iteration % 50 == 0:
        logger.info('Iteration: %i, Time: %e, dt: %e' %(solver.iteration, solver.sim_time, dt))


In [ ]:
from dedalus.tools import post
post.merge_process_files("digital", cleanup=True)